<a href="https://colab.research.google.com/github/Bhimsendabby/Machine--Learning-practice/blob/main/pytorch_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import glob
import time
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb

import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_colab = None

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [4]:
#For numpy gradient
# X = np.array([1, 2, 3,4])
# Y = np.array([2, 4, 6,8])
# w =0.0

learning_rate = 0.01

# def forward(x):
#     return x * 2

# def loss(y_pred, y_true):
#     return ((y_pred - y_true)**2).mean()


#For numpy gradient
# def loss_grad(x, y_pred, y_true):
#     return np.dot(2 * x,(y_pred - y_true)).mean()

#with numpy
# for epoch in range(100):
#     y_pred = forward(X)
#     l = loss(y_pred, Y)
#     grad = loss_grad(X, y_pred, Y)

#     w = w - learning_rate * grad
#     if epoch % 10 == 0:
#         print(f'epoch {epoch + 1}, loss {l:.4f}, weight {w:.4f}')


# print(f'Result: after 100 epochs{forward(10)}')


#with torch

import torch

# Input and output tensors
X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

# Initialize weight
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# Learning rate
learning_rate = 0.01

# Forward pass function
def forward(x):
    return x * w

# Loss function
def loss(y_pred, y_true):
    return ((y_pred - y_true)**2).mean()

# Training loop
for epoch in range(100):
    y_pred = forward(X)
    l = loss(y_pred, Y)
    l.backward()

    with torch.no_grad():
        w -= learning_rate * w.grad

    w.grad.zero_()

    if epoch % 10 == 0:
        print(f'epoch {epoch + 1}, loss {l:.4f}, weight {w:.4f}')

# Test the result
print(f'Result: after 100 epochs, prediction for 10 is {forward(torch.tensor(10.0)):.4f}')

epoch 1, loss 30.0000, weight 0.3000
epoch 11, loss 1.1628, weight 1.6653
epoch 21, loss 0.0451, weight 1.9341
epoch 31, loss 0.0017, weight 1.9870
epoch 41, loss 0.0001, weight 1.9974
epoch 51, loss 0.0000, weight 1.9995
epoch 61, loss 0.0000, weight 1.9999
epoch 71, loss 0.0000, weight 2.0000
epoch 81, loss 0.0000, weight 2.0000
epoch 91, loss 0.0000, weight 2.0000
Result: after 100 epochs, prediction for 10 is 20.0000


In [8]:
loss = nn.MSELoss()

optimizer = optim.SGD([w], lr=learning_rate)

# Training loop
for epoch in range(100):
    y_pred = forward(X)
    l = loss(y_pred, Y)

    l.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 10 == 0:
        print(f'epoch {epoch + 1}, loss {l:.4f}, weight {w:.4f}')

# Test the result
print(f'Result: after 100 epochs, prediction for 10 is {forward(torch.tensor(10.0)):.4f}')

epoch 1, loss 0.0000, weight 2.0000
epoch 11, loss 0.0000, weight 2.0000
epoch 21, loss 0.0000, weight 2.0000
epoch 31, loss 0.0000, weight 2.0000
epoch 41, loss 0.0000, weight 2.0000
epoch 51, loss 0.0000, weight 2.0000
epoch 61, loss 0.0000, weight 2.0000
epoch 71, loss 0.0000, weight 2.0000
epoch 81, loss 0.0000, weight 2.0000
epoch 91, loss 0.0000, weight 2.0000
Result: after 100 epochs, prediction for 10 is 20.0000


In [12]:
#Linear Regression

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32)

num_samples, num_features = X.shape

print(f'Number of samples: {num_samples}')
print(f'Number of features: {num_features}')

# Define the model
input_size = num_features
output_size = 1

model = nn.Linear(input_size, output_size)



print(f'Result: Before epochs, prediction for 10 is {model(X_test).item():.4f}')

Number of samples: 4
Number of features: 1
Result: Before epochs, prediction for 10 is -3.4915


In [15]:
loss = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(100):
    y_pred = model(X)
    l = loss(y_pred, Y)

    l.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 10 == 0:
        w, b = model.parameters()
        print(f'epoch {epoch + 1}, loss {l:.4f}, weight {w[0][0].item():.4f}')

# Test the result
print(f'Result: after 100 epochs, prediction for 10 is {model(X_test).item():.4f}')

epoch 1, loss 0.0001, weight 1.9900
epoch 11, loss 0.0001, weight 1.9903
epoch 21, loss 0.0001, weight 1.9906
epoch 31, loss 0.0001, weight 1.9909
epoch 41, loss 0.0001, weight 1.9912
epoch 51, loss 0.0001, weight 1.9914
epoch 61, loss 0.0001, weight 1.9917
epoch 71, loss 0.0001, weight 1.9919
epoch 81, loss 0.0001, weight 1.9922
epoch 91, loss 0.0001, weight 1.9924
Result: after 100 epochs, prediction for 10 is 9.9848


In [19]:
class LinearRegression(nn.Module):
    def __init__(self, input_size,output_size) -> None:
        super(LinearRegression,self).__init__()

        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

model = LinearRegression(input_size, output_size)



print(f'Result: Before epochs, prediction for 10 is {model(X_test).item():.4f}')

loss = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(100):
    y_pred = model(X)
    l = loss(y_pred, Y)

    l.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 10 == 0:
        w, b = model.parameters()
        print(f'epoch {epoch + 1}, loss {l:.4f}, weight {w[0][0].item():.4f}')

# Test the result
print(f'Result: after 100 epochs, prediction for 10 is {model(X_test).item():.4f}')

Result: Before epochs, prediction for 10 is -0.2651
epoch 1, loss 31.5608, weight 0.2522
epoch 11, loss 0.8788, weight 1.5438
epoch 21, loss 0.0813, weight 1.7567
epoch 31, loss 0.0573, weight 1.7960
epoch 41, loss 0.0534, weight 1.8072
epoch 51, loss 0.0503, weight 1.8137
epoch 61, loss 0.0474, weight 1.8193
epoch 71, loss 0.0446, weight 1.8247
epoch 81, loss 0.0420, weight 1.8299
epoch 91, loss 0.0396, weight 1.8349
Result: after 100 epochs, prediction for 10 is 9.6690
